In [1]:
from gensim.models import KeyedVectors
import numpy as np
from joblib import dump, load
import pandas as pd

In [2]:
# Usa frecuencia de palabras
def create_tf_idf(list_text):
	vectorizer = TfidfVectorizer(use_idf=False, norm=None, ngram_range=(1, 1)) # use_idf=true ponderacion de frecuancia de palabras en documentos
	tfidf = vectorizer.fit(list(list_text))
	vectores_tfidf=vectorizer.transform(list_text)
	dump(tfidf,'tfidf_model.joblib')
	print('Fue creado modelo TF-IDF')
	#vectores_tfidf=list(matrix)
	#print(len(vectores_tfidf))
	return vectores_tfidf

In [3]:
# Retorna un vector TF-IDF 
def get_vector_tfidf(list_text):
	tfidf=load('tfidf_model.joblib')# Ojo cuidado, solo cargar solo una vez
	words_ntf=[x for x in list_text.split() if x not in tfidf.get_feature_names()] # Palabras que no estan en el vocabulario del modelo
	return tfidf.transform([list_text]).toarray()

In [4]:
data_g = pd.read_csv('natacion_suicidio.csv') 

In [5]:
clase=list(data_g.suicidio)
text=list(data_g.tweet_clean)

In [6]:
selec_dat=data_g.loc[data_g['suicidio'] == 1]

In [7]:
selec_dat.head(5)

,frase,tweet_clean,suicidio
7,cansado de estar solo,louis espero no estar pidiendo mucho solo quie...,1
17,cansado de estar solo,estoy super cansado cansado de la gente cansad...,1
26,dormir y nunca mas despertar,dormir y desear nunca más despertar,1
27,dormir y nunca mas despertar,ojalá dormir y nunca mas despertar,1
29,dormir y nunca mas despertar,me cansé de todo no tengo ganas ni de ir al ba...,1


In [8]:
selec_data_text=list(selec_dat.tweet_clean)
selec_data_tipo=list(selec_dat.frase)

In [9]:
data_todo = pd.read_csv('suicidio_todo.csv') 

In [10]:
text_todo=list(data_todo.tweet_clean.values.astype('U'))
text_todo_tipo=list(data_todo.frase.values.astype('U'))

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import KeyedVectors

In [12]:
vectores_tfidf=create_tf_idf(text_todo)

Fue creado modelo TF-IDF


In [13]:
vectores_tfidf.shape

(54487, 47585)

In [14]:
import math
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
vec_sele=get_vector_tfidf(j)
vec_todo_j=get_vector_tfidf(y)

In [22]:
for i,j in zip(selec_data_tipo, vec_sele):
    #vec_sele=get_vector_tfidf(j)
    for x,y in zip(text_todo_tipo, vec_todo_j):
        #vec_todo_j=get_vector_tfidf(y)
        print(i,'--',x)
        if i==x:
            simi=cosine_similarity(vec_sele,vec_todo_j)[0][0]
            print(simi)

cansado de estar solo -- dormir y nunca mas despertar
